<h2>Web Crawler</h3>
In general terms, the term
<b>“crawler”</b> indicates a program’s ability to navigate web pages on its own, perhaps even
without a well-defined end goal or purpose, endlessly exploring what a site or the web has
to offer. While <b>“scraping”</b>  indicates a program that is extracting information from pages. 

As our first example, we’re going to work with the page at <a>http://www.
webscrapingfordatascience.com/crawler/</a>. This page tries to emulate a simple
numbers station (basically spit out a random list of numbers and links to
direct you to a different page.),
<br>
Note that we’re using the urljoin function here. The reason why we do so is because
the “href” attribute of links on the page refers to a relative URL, for instance, “?r=f01e7f
02e91239a2003bdd35770e1173”, which we need to convert to an absolute one. We could
just do this by prepending the base URL, that is, base_url + link_url, but once we’d
start to follow links and pages deeper in the site’s URL tree, that approach would fail to
work. 

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
base_url = 'http://www.webscrapingfordatascience.com/crawler/'
links_seen = set() #can't allow duplicates set like math set
def visit(url, links_seen):
    html = requests.get(url).text
    html_soup = BeautifulSoup(html, 'html.parser')
    links_seen.add(url)
    for link in html_soup.find_all("a"):
        link_url = link.get('href')
        if link_url is None:
            continue
        # print(link_url) #?r=d494086c12004b50f5bcaf20dcd95cec !realative path 
        full_url = urljoin(url, link_url)
        if full_url in links_seen:
            continue
        print('Found a new page:', full_url)
        # Normally, we'd store the results here too
        visit(full_url, links_seen)
visit(base_url, links_seen)

?r=d494086c12004b50f5bcaf20dcd95cec


if you run above script, you’ll see that it will start visiting the different URLs. If you let it
run for a while, however, this script will certainly crash, and not only because of network
hiccups. The reason for this is because we use <b>recursion</b>: the visit function is calling
itself over and over again, without an opportunity to go back up in the call tree as every
page will contain links to other pages. relying on recursion for web crawling is generally not a robust idea. We can
rewrite our code as follows without recursion:

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
links_todo = ['http://www.webscrapingfordatascience.com/crawler/']
links_seen = set()
def visit(url, links_seen):
    html = requests.get(url).text
    html_soup = BeautifulSoup(html, 'html.parser')
    new_links = []
    for link in html_soup.find_all("a"):
        link_url = link.get('href')
        if link_url is None:
            continue
        full_url = urljoin(url, link_url)
        if full_url in links_seen:  
            continue
        # Normally, we'd store the results here too
        new_links.append(full_url)
    return new_links
while links_todo:
    url_to_visit = links_todo.pop()
    links_seen.add(url_to_visit)
    print('Now visiting:', url_to_visit)
    new_links = visit(url_to_visit, links_seen)
    print(len(new_links), 'new link(s) found')
    links_todo += new_links

Above program is better, but it still has several
drawbacks. If our program crashes (e.g., when your Internet connection or the website
is down), you’ll have to restart from scratch again. Also, we have no idea how large the
links_seen set might become. Normally, your computer will have plenty of memory
available to easily store thousands of URLs, though we might wish to resort to a database
to store intermediate progress information as well as the results

<h3>Storing Results in DB</h3>
Let’s adapt our example to make it more robust against crashes by storing progress and
result information in a database. We’re going to use the “records” library to manage an
SQLite database (a file based though powerful database system) in which we’ll store our
queue of links and retrieved numbers from the pages we crawl, which can be installed
using pip:
<i>pip install -U records</i>
<br>
<b>WORKED for Mysql</b>

In [ ]:
import requests

import mysql.connector as mysql
from mysql.connector import Error #error library for error messages 
from bs4 import BeautifulSoup
from urllib.parse import urljoin
conn=mysql.connect(host='localhost',database='web_crawler',user='root',password='')
db=conn.cursor()
# db.execute('DROP TABLE IF EXISTS links')
# db.execute('DROP TABLE IF EXISTS numbers')

db.execute('''CREATE TABLE IF NOT EXISTS links (
url varchar(500) PRIMARY KEY,
created_at datetime,
visited_at datetime NULL)''')
db.execute('''CREATE TABLE IF NOT EXISTS numbers (url varchar(500), number integer,
PRIMARY KEY (url, number))''')

def store_link(url):
    try:
        db.execute('''INSERT INTO links (url, created_at)
        VALUES (\'%s\', CURRENT_TIMESTAMP)'''%url)
        conn.commit()
    except Error as e:
        print('Inserting URL',format(e))

def get_random_unvisited_link():
    db.execute('SELECT * FROM links WHERE visited_at IS NULL ORDER BY RAND() LIMIT 1')
    data=db.fetchone()
    url=data[0]
    return None if data is None else url

def store_number(url, number):
    try:
        db.execute('''INSERT INTO numbers (url, number)
        VALUES (\'%s\', %d)'''%(url,number))
    except Error as e: 
        print('Storing Number',format(e))

def visit(url):
    html = requests.get(url).text
    html_soup = BeautifulSoup(html, 'html.parser')
    new_links = []
    for td in html_soup.find_all("td"):
        store_number(url, int(td.text.strip()))
    for link in html_soup.find_all("a"):
        link_url = link.get('href')
        if link_url is None:continue
        full_url = urljoin(url, link_url)
        new_links.append(full_url)
    return new_links

def mark_visited(url):
    db.execute('''UPDATE links SET visited_at=CURRENT_TIMESTAMP
WHERE url=\'%s\'''' %url)
    conn.commit()

store_link('http://www.webscrapingfordatascience.com/crawler/')
url_to_visit = get_random_unvisited_link()

while url_to_visit is not None:
    print('Now visiting:', url_to_visit)
    new_links = visit(url_to_visit)
    print(len(new_links), 'new link(s) found')
    for link in new_links:
        store_link(link)
    mark_visited(url_to_visit)
    url_to_visit = get_random_unvisited_link()
    

Let’s now try to use the same framework in order to build a crawler for Wikipedia. Our
plan here is to store page titles, as well as keep track of “(from, to)” links on each page,
starting from the main page. Note that our database scheme looks a bit different here: <b> WORKED FOR mysql</b>

In [19]:
import requests

import mysql.connector as mysql
from mysql.connector import Error #error library for error messages 
from bs4 import BeautifulSoup
from urllib.parse import urljoin,urldefrag
conn=mysql.connect(host='localhost',database='wikipedia',user='root',password='')
db=conn.cursor()
# db.execute('DROP TABLE IF EXISTS links')
# db.execute('DROP TABLE IF EXISTS numbers')
base_url = 'https://en.wikipedia.org/wiki/'

def create_tables():
    db.execute('''CREATE TABLE IF NOT EXISTS pages
(url varchar(500) PRIMARY KEY ,
page_title text NULL,
created_at datetime, 
visited_at datetime NULL)''')
    db.execute('''CREATE TABLE IF NOT EXISTS links (
url varchar(255) ,
url_to varchar(255),
PRIMARY KEY(url,url_to ))''')

def store_page(url):
    try:
        db.execute('''INSERT INTO pages (url, created_at)
        VALUES (\'%s\', CURRENT_TIMESTAMP)'''%url)
        conn.commit()
    except Error as e:
        print('Inserting URL',format(e))

def get_random_unvisited_page():
    db.execute('SELECT * FROM pages WHERE visited_at IS NULL ORDER BY RAND() LIMIT 1')
    data=db.fetchone()
    url=data[0]
    return None if data is None else url

def set_title(url, page_title):
    page_title=page_title.replace('\'','')
    print(page_title)
    db.execute('UPDATE pages SET page_title=\'%s\' WHERE url=\'%s\''%(page_title,url))
    conn.commit()

def store_link(url, url_to):
    try:
        db.execute('''INSERT INTO links (url, url_to)
        VALUES (\'%s\',\'%s\')'''%(url,url_to))
        conn.commit()
    except Error as ie:
        print(format(ie))

def set_visited(url):
    db.execute('''UPDATE pages SET visited_at=CURRENT_TIMESTAMP
    WHERE url=\'%s\''''%url)
    conn.commit()

def visit(url):
    print('Now visiting:', url)
    resp = requests.get(url)
    html=resp.text
    html_soup = BeautifulSoup(html, 'html.parser')
    page_title = html_soup.find(id='firstHeading')
    page_title = page_title.text if page_title else ''
    print(' page title:', page_title)
    print('Response: ',resp.url)
    print(url)
    print()
    set_title(url, page_title)
    for link in html_soup.find_all("a"):
        link_url = link.get('href')
        if link_url is None:continue
        full_url = urljoin(base_url, link_url)
        # Remove the fragment identifier part
        full_url = urldefrag(full_url)[0]
        if not full_url.startswith(base_url):continue # This is an external link, skip
        store_link(url, full_url)
        store_page(full_url)
    set_visited(url)

create_tables()
store_page(base_url)
url_to_visit = get_random_unvisited_page()
print(url_to_visit)
while url_to_visit is not None:
    visit(url_to_visit)
    url_to_visit = get_random_unvisited_page()


In above program we know in advance which information we want
to get out of the pages (the page title and links, in this case). In case you don’t know
beforehand what you’ll want to get out of crawled pages, you might want to split up the
crawling and parsing of pages even further, for example, by storing a complete copy
of the HTML contents that can be parsed by a second script. 

In [28]:
import requests
import re
import os, os.path
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urldefrag
import mysql.connector as mysql
from mysql.connector import Error #error library for error messages 

conn=mysql.connect(host='localhost',database='wikipedia_2',user='root',password='')
db=conn.cursor()
# db.execute('DROP TABLE IF EXISTS links')
# db.execute('DROP TABLE IF EXISTS numbers')

def create_tables():
        # This table keeps track of crawled and to-crawl pages
    db.execute('''CREATE TABLE IF NOT EXISTS pages (
    url varchar(500) PRIMARY KEY,
    created_at datetime,
    html_file text NULL,
    visited_at datetime NULL)''')
    # This table keeps track of a-tags
    db.execute('''CREATE TABLE IF NOT EXISTS links (
    url varchar(255), link_url varchar(255),
    PRIMARY KEY (url, link_url))''')
    # This table keeps track of img-tags
    db.execute('''CREATE TABLE IF NOT EXISTS images (
    url varchar(255), img_url varchar(255), img_file text,
    PRIMARY KEY (url, img_url))''')

base_url = 'https://en.wikipedia.org/wiki/'
file_store = './downloads/'


if not os.path.exists(file_store):
    os.makedirs(file_store)

def store_page(url):
    try:
        db.execute('''INSERT INTO pages (url, created_at)
        VALUES (\'%s\', CURRENT_TIMESTAMP)'''%url)
        conn.commit()
    except Error as e:
        pass 
def get_random_unvisited_page():
    db.execute('SELECT * FROM pages WHERE visited_at IS NULL ORDER BY RAND() LIMIT 1')
    data=db.fetchone()
    url=data[0]
    return None if data is None else url

def store_link(url, link_url):
    try:
        db.execute('''INSERT INTO links (url, link_url)
        VALUES (\'%s\', \'%s\')'''%(url,link_url))
        conn.commit()
    except Error as ie:
        pass

def should_visit(link_url):
    link_url = urldefrag(link_url)[0]
    if not link_url.startswith(base_url):return None
    return link_url

def url_to_file_name(url):
    url = str(url).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', url)

def download(url, filename):
    r = requests.get(url, stream=True)
    with open(os.path.join(file_store, filename), 'wb') as the_image:
        for byte_chunk in r.iter_content(chunk_size=4096*4):
            the_image.write(byte_chunk)

def store_image(url, img_url, img_file):
    try:
        db.execute('''INSERT INTO images (url, img_url, img_file)
        VALUES (\'%s\', \'%s\', \'%s\')'''%(url,img_url,img_file))
        conn.commit()
    except Error as ie:
        pass
def set_visited(url, html_file):
    db.execute('''UPDATE pages
    SET visited_at=CURRENT_TIMESTAMP,
    html_file=\'%s\'
    WHERE \'%s\'
    '''%(html_file,url))

    conn.commit()
def visit(url):
    print('Now visiting:', url)
    resp = requests.get(url)
    html=resp.text
    html_soup = BeautifulSoup(html, 'html.parser')
    for link in html_soup.find_all("a"):
        link_url = link.get('href')
        if link_url is None:continue
        link_url = urljoin(base_url, link_url)
        store_link(url, link_url)
        full_url = should_visit(link_url)
        if full_url:
            # Queue for crawling
            store_page(full_url)
    for img in html_soup.find_all("img"):
        img_url = img.get('src')
        if img_url is None:continue
        img_url = urljoin(base_url, img_url)
        filename = url_to_file_name(img_url)
        download(img_url, filename)
        store_image(url, img_url, filename)

    # Store the HTML contents
    filename = url_to_file_name(url) + '.html'
    fullname = os.path.join(file_store, filename)
    with open(fullname, 'w', encoding='utf-8') as the_html:
        the_html.write(html)
    set_visited(url, filename)
    
create_tables()   
store_page(base_url)
url_to_visit = get_random_unvisited_page()
while url_to_visit is not None:
    visit(url_to_visit)
    url_to_visit = get_random_unvisited_page()

Now visiting: https://en.wikipedia.org/wiki/Treasurer_of_the_United_States
Now visiting: https://en.wikipedia.org/wiki/Freedman%27s_Bank_Building
Now visiting: https://en.wikipedia.org/wiki/Illusion_of_Kate_Moss
Now visiting: https://en.wikipedia.org/wiki/Michael_Hillegas
Now visiting: https://en.wikipedia.org/wiki/National_Law_Enforcement_Officers_Memorial
Now visiting: https://en.wikipedia.org/wiki/JSTOR_(identifier)
Now visiting: https://en.wikipedia.org/wiki/National_Register_of_Historic_Places_listings_in_New_Hampshire
Now visiting: https://en.wikipedia.org/wiki/Richmond,_New_Hampshire
Now visiting: https://en.wikipedia.org/wiki/Special:RecentChangesLinked/JSTOR
Now visiting: https://en.wikipedia.org/wiki/National_Register_of_Historic_Places_listings_in_Alaska


OSError: [Errno 22] Invalid argument: './downloads/httpsupload.wikimedia.orgwikipediacommonsthumbccaAlaska_Native_Brotherhood_Hall2C_Sitka_Camp_No._12C_Katlian_Street2C_Sitka2C_28Sitka_Borough2C_Alaska29.jpg220px-Alaska_Native_Brotherhood_Hall2C_Sitka_Camp_No._12C_Katlian_Street2C_Sitka2C_28Sitka_Borough2C_Alaska29.jpg'